# Domain Modeling by Kotlin

In [1]:
%use test, serialization

In [2]:
import kotlin.test.Test
import kotlin.test.assertEquals

/** Custom test runner */
fun test(instance: Any) {
    val klass: kotlin.reflect.KClass<out Any> = instance::class
    val RED_BOLD = "\u001B[1;31m"
    val GREEN_BOLD = "\u001B[1;32m"
    val RED = "\u001B[0;31m"    
    val RESET = "\u001B[0m"
    
    fun filterStack(line: String): Boolean {
        return (
            line.indexOf("TestRunner.run") < 0 && line.indexOf("_jupyter.") < 0 &&
            line.startsWith("at Line")
        )
    }
    
    klass.members.filter { 
        it.annotations.size > 0 && 
        it.annotations.elementAt(0).annotationClass == org.junit.Test::class
    }.forEach {
        try {
            it.call(instance)
            println("✅ ${GREEN_BOLD}${it.name}${RESET}")
        } catch(e: java.lang.reflect.InvocationTargetException) {
            println("❌ ${RED}${it.name}${RESET}")
            val cause = e.cause.toString()
            val idx = cause.indexOf(":")
            val (name, reason) = Pair(cause.substring(0, idx), cause.substring(idx+1))
            println("${name}:${RED_BOLD}${reason}${RESET}")
            val trace = e.stackTraceToString().split('\n').filter { filterStack(it.trim()) }
            println(trace.joinToString("\n") + "\n")
        }
    }
}

In [3]:
import java.time.LocalDate
import kotlinx.serialization.*
import kotlinx.serialization.json.*
import kotlinx.serialization.internal.*

@Serializable
data class Batch (
    val reference: String,
    val sku: String,
    val qty: Int,
    val eta: LocalDate,
    val allocations: MutableList<OrderLine> = mutableListOf()
) {
    val allocatedQty get() = allocations.map { it.qty }.sum()
    val availableQty get() = (qty - allocatedQty)
    
    fun canAllocate(line: OrderLine): Boolean {
        return sku == line.sku && availableQty >= line.qty
    }
    
    fun allocate(line: OrderLine) {
        if (canAllocate(line)) {
            allocations.add(line)
        }
    }
    
    fun deallocate(line: OrderLine) {
        if (allocations.contains(line))
            allocations.remove(line)
    }
}

@Serializable
data class OrderLine (
    val id: String,
    val sku: String,
    val qty: Int
)

@Serializable
data class Order (
    val reference: String,
    val orderLines: List<OrderLine> = listOf()
)

In [4]:
class Chapter1Test {
    val today = LocalDate.now()
    val tomorrow = today.plusDays(1)
   
    fun makeBatchAndLine(sku: String, batchQty: Int, lineQty: Int): Pair<Batch, OrderLine> {
        return Pair(
            Batch("batch-001", sku, batchQty, eta=today),
            OrderLine("order-123", sku, lineQty)
        )
    }
    
    @Test 
    fun `Test allocating to a batch reduces the available quantity`() {
        val batch = Batch("batch-001", "SMALL-TABLE", qty=20, eta=today)
        val line = OrderLine("order-ref", "SMALL-TABLE", 2)
        batch.allocate(line)
        assertEquals(18, batch.availableQty)
    }
    
    @Test
    fun `Test can allocate if available greater than required`() {
        val (largeBatch, smallLine) = makeBatchAndLine("ELEGANT-LAMP", 20, 2)
        assertTrue(!largeBatch.canAllocate(smallLine))
    }
}

test(Chapter1Test())

✅ Test allocating to a batch reduces the available quantity
❌ Test can allocate if available greater than required
java.lang.AssertionError: Expected value to be true.
	at Line_4_jupyter$Chapter1Test.Test can allocate if available greater than required(Line_4.jupyter.kts:23)

